In [1]:
import scrapy
import json
import re
from scrapy.crawler import CrawlerProcess
from scrapper import gameScrapSpider, run_spider

run_spider()

2021-11-22 09:30:08 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-11-22 09:30:08 [scrapy.utils.log] INFO: Versions: lxml 4.6.4.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.9.9 (tags/v3.9.9:ccb0e6a, Nov 15 2021, 18:08:50) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 35.0.0, Platform Windows-10-10.0.19042-SP0
2021-11-22 09:30:08 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-11-22 09:30:08 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-11-22 09:30:08 [scrapy.utils.log] INFO: Versions: lxml 4.6.4.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.9.9 (tags/v3.9.9:ccb0e6a, Nov 15 2021, 18:08:50) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 35.0.0, Platform Windows-10-10.0.19042-SP0
2021-11-22 09:30:08 [scrapy.crawler] INFO: Overridden se

2021-11-22 09:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.promobit.com.br/promocoes/jogos-de-pc/s/>
{'name': ['Jogo Call of Duty: Black Ops Cold War - PC Battle.net'], 'store': ['battle.net'], 'price': ['R$\xa0229,90'], 'discounted price': ['114,90']}
2021-11-22 09:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.promobit.com.br/promocoes/jogos-de-pc/s/>
{'name': ['Jogo BioShock Infinite - PC Steam'], 'store': ['Steam'], 'price': ['R$\xa089,99'], 'discounted price': ['22,49']}
2021-11-22 09:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.promobit.com.br/promocoes/jogos-de-pc/s/>
{'name': ["Jogo Deus Ex: Human Revolution Director's Cut - PC Steam"], 'store': ['Steam'], 'price': ['R$\xa034,99'], 'discounted price': ['5,24']}
2021-11-22 09:30:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.promobit.com.br/promocoes/jogos-de-pc/s/>
{'name': ['Jogo Deus Ex: The Fall - PC Steam'], 'store': ['Steam'], 'price': ['R$\xa016,99']

In [2]:
def cleanupTitle(full_title):
    splitlist = re.split("Jogo | -", full_title)
    return splitlist[1]

def cleanupFullPrice(price):
    splitlist = price.split("\xa0")
    return splitlist[-1]

In [3]:
with open ('items.json') as file:
    data = json.load(file)

games_by_store = {}
for item in data:
    prices = {}
    full_title = item['name'][0]
    store = item['store'][0]
    notclean_price = item['price'][0]
    discounted_price = item['discounted price'][0]
    
    game = cleanupTitle(full_title)
    full_price = cleanupFullPrice(notclean_price)
    prices = [full_price, discounted_price]
    
    if store not in games_by_store.keys():
        games_by_store[store] = []
    games_by_store[store].append({'game': game, 'prices': prices})

for item in games_by_store:
    print(item)
    print(games_by_store[item])

gog.com
[{'game': 'Cyberpunk 2077', 'prices': ['199,90', '99,99']}]
gamersgate.com
[{'game': 'Red Dead Redemption II', 'prices': ['239,00', '107,55']}]
Steam
[{'game': 'Empire of Sin', 'prices': ['75,49', '37,74']}, {'game': 'Just Cause 3', 'prices': ['42,99', '9,58']}, {'game': 'Space Engineers', 'prices': ['37,99', '26,59']}, {'game': 'BioShock Infinite', 'prices': ['89,99', '22,49']}, {'game': "Deus Ex: Human Revolution Director's Cut", 'prices': ['34,99', '5,24']}, {'game': 'Deus Ex: The Fall', 'prices': ['16,99', '3,39']}, {'game': 'Deus Ex: Invisible War', 'prices': ['12,99', '1,81']}, {'game': "The Witch's House MV", 'prices': ['28,99', '14,49']}, {'game': 'Minoria', 'prices': ['37,99', '18,99']}]
battle.net
[{'game': 'Call of Duty: Black Ops Cold War', 'prices': ['229,90', '114,90']}]
gog.com
[{'game': 'Cyberpunk 2077', 'prices': ['199,90', '99,99']}]
gamersgate.com
[{'game': 'Red Dead Redemption II', 'prices': ['239,00', '107,55']}]
Steam
[{'game': 'Empire of Sin', 'prices': [